## Load the digits dataset and evolutionary_search

In [2]:
from evolutionary_search import EvolutionaryAlgorithmSearchCV
import sklearn.datasets
import numpy as np

data = sklearn.datasets.load_digits()
X = data["data"]
y = data["target"]

# make it a 2-class problem by only classifying the digit "5" vs the rest
y = np.array([1 if yy == 5 else 0 for yy in y])

X.shape, y.shape

((1797, 64), (1797,))

## Train an SVM with RBF kernel

### Using conventional GridSearchCV

Parameter grid: 625 parameter combinations

In [3]:
paramgrid = {"kernel": ["rbf"],
             "C"     : np.logspace(-9, 9, num=25, base=10),
             "gamma" : np.logspace(-9, 9, num=25, base=10)}

In [4]:
from sklearn.cross_validation import StratifiedKFold
from sklearn.grid_search import GridSearchCV
from sklearn.svm import SVC

cv = GridSearchCV(estimator=SVC(),
                  param_grid=paramgrid,
                  scoring="accuracy",
                  cv=StratifiedKFold(y, n_folds=10),
                  verbose=1)
%time cv.fit(X, y)

[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  50 jobs       | elapsed:    3.3s
[Parallel(n_jobs=1)]: Done 200 jobs       | elapsed:   12.7s
[Parallel(n_jobs=1)]: Done 450 jobs       | elapsed:   28.3s
[Parallel(n_jobs=1)]: Done 800 jobs       | elapsed:   50.1s
[Parallel(n_jobs=1)]: Done 1250 jobs       | elapsed:  1.3min
[Parallel(n_jobs=1)]: Done 1800 jobs       | elapsed:  1.9min
[Parallel(n_jobs=1)]: Done 2450 jobs       | elapsed:  3.0min
[Parallel(n_jobs=1)]: Done 3200 jobs       | elapsed:  5.7min
[Parallel(n_jobs=1)]: Done 4050 jobs       | elapsed:  9.0min
[Parallel(n_jobs=1)]: Done 5000 jobs       | elapsed: 12.7min
[Parallel(n_jobs=1)]: Done 6050 jobs       | elapsed: 16.5min


Fitting 10 folds for each of 625 candidates, totalling 6250 fits
CPU times: user 17min 15s, sys: 13 s, total: 17min 28s
Wall time: 17min 30s


[Parallel(n_jobs=1)]: Done 6250 out of 6250 | elapsed: 17.5min finished


GridSearchCV(cv=sklearn.cross_validation.StratifiedKFold(labels=[0 0 ..., 0 0], n_folds=10, shuffle=False, random_state=None),
       error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0, degree=3, gamma=0.0,
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False),
       fit_params={}, iid=True, loss_func=None, n_jobs=1,
       param_grid={'kernel': ['rbf'], 'C': array([  1.00000e-09,   5.62341e-09,   3.16228e-08,   1.77828e-07,
         1.00000e-06,   5.62341e-06,   3.16228e-05,   1.77828e-04,
         1.00000e-03,   5.62341e-03,   3.16228e-02,   1.77828e-01,
         1.00000e+00,   5.62341e+00,   3.16228e+01,   1.77828e+...7828e+05,
         1.00000e+06,   5.62341e+06,   3.16228e+07,   1.77828e+08,
         1.00000e+09])},
       pre_dispatch='2*n_jobs', refit=True, score_func=None,
       scoring='accuracy', verbose=1)

### Best score + params

In [5]:
cv.best_score_, cv.best_params_

(0.99443516972732326, {'C': 1.0, 'gamma': 0.001, 'kernel': 'rbf'})

### Using RandomizedSearchCV

Same parameter space, but only test 250 random combinations.

In [6]:
paramgrid = {"kernel": ["rbf"],
             "C"     : np.logspace(-9, 9, num=100, base=10),
             "gamma" : np.logspace(-9, 9, num=100, base=10)}

In [7]:
from sklearn.grid_search import RandomizedSearchCV

cv = RandomizedSearchCV(estimator=SVC(),
                        param_distributions=paramgrid,
                        n_iter=250,
                        scoring="accuracy",
                        cv=StratifiedKFold(y, n_folds=10),
                        verbose=1)
%time cv.fit(X, y)

[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  50 jobs       | elapsed:   11.5s
[Parallel(n_jobs=1)]: Done 200 jobs       | elapsed:   38.8s
[Parallel(n_jobs=1)]: Done 450 jobs       | elapsed:  1.3min
[Parallel(n_jobs=1)]: Done 800 jobs       | elapsed:  2.3min
[Parallel(n_jobs=1)]: Done 1250 jobs       | elapsed:  3.7min
[Parallel(n_jobs=1)]: Done 1800 jobs       | elapsed:  5.1min
[Parallel(n_jobs=1)]: Done 2450 jobs       | elapsed:  6.9min


Fitting 10 folds for each of 250 candidates, totalling 2500 fits
CPU times: user 7min 5s, sys: 5.25 s, total: 7min 10s
Wall time: 7min 11s


[Parallel(n_jobs=1)]: Done 2500 out of 2500 | elapsed:  7.2min finished


RandomizedSearchCV(cv=sklearn.cross_validation.StratifiedKFold(labels=[0 0 ..., 0 0], n_folds=10, shuffle=False, random_state=None),
          error_score='raise',
          estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0, degree=3, gamma=0.0,
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False),
          fit_params={}, iid=True, n_iter=250, n_jobs=1,
          param_distributions={'kernel': ['rbf'], 'C': array([  1.00000e-09,   1.51991e-09, ...,   6.57933e+08,   1.00000e+09]), 'gamma': array([  1.00000e-09,   1.51991e-09, ...,   6.57933e+08,   1.00000e+09])},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          scoring='accuracy', verbose=1)

### Best score + params

In [8]:
cv.best_score_, cv.best_params_

(0.99443516972732326,
 {'C': 5336.6992312063239, 'gamma': 0.0015199110829529332, 'kernel': 'rbf'})

### Using EvolutionaryAlgorithmSearchCV

Again same parameter space, optimize for 10 generations.

In [9]:
paramgrid = {"kernel": ["rbf"],
             "C"     : np.logspace(-9, 9, num=100, base=10),
             "gamma" : np.logspace(-9, 9, num=100, base=10)}

In [10]:
cv = EvolutionaryAlgorithmSearchCV(estimator=SVC(),
                                   param_grid=paramgrid,
                                   scoring="accuracy",
                                   cv=StratifiedKFold(y, n_folds=10),
                                   verbose=True,
                                   population_size=50,
                                   mutation_prob=0.10,
                                   tournament_size=3,
                                   generations_number=10)
%time cv.fit(X, y)

--- Evolve in 10000 possible combinations ---
gen	nevals	avg    	min	max     
0  	50    	0.62391	0  	0.992762
1  	38    	0.843243	0  	0.994438
2  	23    	0.933237	0.898726	0.994438
3  	29    	0.902628	0       	0.994438
4  	32    	0.846655	0       	0.994438
5  	22    	0.975516	0.898726	0.994438
6  	30    	0.972667	0.898726	0.994438
7  	26    	0.986291	0.898726	0.994438
8  	36    	0.960804	0       	0.994438
9  	33    	0.914839	0       	0.994438
10 	27    	0.972235	0       	0.994438
Best individual is: {"kernel": "rbf", "C": 432876128.10830617, "gamma": 0.0015199110829529332}
with fitness: (0.9944381342359675,)
CPU times: user 4min 32s, sys: 3.35 s, total: 4min 35s
Wall time: 4min 35s


### Best score + params

In [11]:
cv.best_score_, cv.best_params_

(deap.creator.FitnessMax((0.9944381342359675,)),
 {'C': 432876128.10830617, 'gamma': 0.0015199110829529332, 'kernel': 'rbf'})